正常拟合、欠拟合和过拟合（正常为三阶多项式）

In [ ]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

max_degree = 20  # 设置最大次数
n_train, n_test = 100,100
true_w = np.zeros(max_degree)
true_w[0:4] = np.array([5,1.2,-3.4,5.6])  # 权重

features = np.random.normal(size=(n_test+n_train,1))
np.random.shuffle(features)  # 随机打乱 features中数据

# 生成多项式
poly_features = np.power(features, np.arange(max_degree).reshape(1,-1))
#               进行幂运算          生成0-19的数值数组，对应次数   转化为行向量方便幂运算

for i in range(max_degree):
    poly_features[:,i] /= math.gamma(i+1)
labels = np.dot(poly_features,true_w)
labels += np.random.normal(scale=0.1,size=labels.shape)

true_w, features, poly_features, labels = [torch.tensor(x,dtype=torch.float32) for x in [true_w, features, poly_features, labels]]

# 实现一个函数来评估模型在给定数据集上的损失
def evaluate_loss(net, data_iter, loss):
    metric = d2l.Accumulator(2)  # 两个数的累加器
    for X, y in data_iter:  # 从迭代器中拿出对应特征和标签
        out = net(X)
        y = y.reshape(out.shape)  # 将真实标签改为网络输出标签的形式，统一形式
        l = loss(out, y)  # 计算网络输出的预测值与真实值之间的损失差值
        metric.add(l.sum(), l.numel())  # 总量除以个数，等于平均
    return metric[0] / metric[1]  # 返回数据集的平均损失

# 训练函数
def train(train_features, test_features, train_labels, test_labels, num_epochs=400):
    loss = nn.MSELoss()
    input_shape = train_features.shape[-1]
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False)) # 单层线性回归
    batch_size = min(10,train_labels.shape[0])
    train_iter = d2l.load_array((train_features,train_labels.reshape(-1,1)),batch_size)
    test_iter = d2l.load_array((test_features,test_labels.reshape(-1,1)),batch_size,is_train=False)
    trainer = torch.optim.SGD(net.parameters(),lr=0.01)
    animator = d2l.Animator(xlabel='epoch',ylabel='loss',yscale='log',xlim=[1,num_epochs],ylim=[1e-3,1e2],legend=['train','test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss), evaluate_loss(net,test_iter,loss)))
    print('weight',net[0].weight.data.numpy()) # 训练完后打印，打印最终学到的weight值

# 正常拟合
# train(poly_features[:n_train,:4],poly_features[n_train:,:4],labels[:n_train],labels[n_train:])
# 欠拟合 ——用一阶多项式拟合真实的三阶多项式
# train(poly_features[:n_train,:2],poly_features[n_train:,:2],labels[:n_train],labels[n_train:])
# 过拟合 ——用十九阶多项式拟合真实的三阶多项式
# train(poly_features[:n_train,:],poly_features[n_train:,:],labels[:n_train],labels[n_train:])

d2l.plt.show()